In [1]:
from google.cloud import bigquery 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from google.cloud import bigquery 
import json
import os
from datetime import datetime, timedelta , date 
import snscrape.modules.twitter as sntwitter
import re
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
#May have some issues running this code to retrieve old data due to API issues by snscrape library
def mapper(key):
    if key == 'grab':
        return '#grab'
    elif key =='dbs':
        return 'dbs AND bank'
    else:
        return key
# Specify the search criteria
tickers=[
    ('d05.SI', 'dbs'),
    ('SE', 'garena'),
    ('SE', 'shopee'),
    ('039.SI', 'ocbc'),
    ('U11.SI', 'uob'),
    ('Z74.SI', 'singtel'),
    ('F34.SI', 'wilmar'),
    ('C6L.SI', 'singapore airlines'),
    ('GRAB', 'grab'),
    ('G13.SI', 'genting'),
    ('C38U.SI', 'capitaland'),
    ('G07.SI', 'great eastern'),
    ('C07.SI', 'jardine'),
    ('A17U.SI', 'ascendas'),
    ('S63.SI', 'st engineering'),
    ('BN4.SI', 'keppel')
]


# Create an empty list to store the tweets
tweets = []
for tup in tickers:
    keyword = mapper(tup[1])
     #can use OR to specify other things #hashtag also can
    start_date = date(2022, 2, 18)
    end_date = date(2023, 4, 19)
    user = ''

    # Create a list of dates between start_date and end_date
    delta = timedelta(days=1)
    date_range = pd.date_range(start_date, end_date, freq='D').tolist()


    #limit of tweets per day
    limit = 100

    # Loop over each day in the date range and retrieve X tweets per day
    #if count == limit or when the loop ends
    for i in range(len(date_range)):
        day = date_range[i].strftime('%Y-%m-%d')
        dayNext = (date_range[i] + timedelta(days = 1)).strftime('%Y-%m-%d')
        query = f'{keyword} since:{day} until:{dayNext} lang:en'
        count = 0
        for tweet in sntwitter.TwitterSearchScraper(query).get_items():
            if (count == limit):
                break
            tweets.append([tup[1], tup[0], tweet.date, tweet.content, tweet.likeCount])
            count+= 1
        print(f'{tup[1]}_{day}: {count} total tweets retrieved')

# Convert the list of tweets to a Pandas DataFrame
rawData = pd.DataFrame(tweets, columns=['Ticker_Name', 'Ticker', 'Datetime', 'Text', 'Like_Count'])

In [2]:
def clean_tweet(tweet):
        # method for cleaning tweets used in clean_twitter_data dag

        tweet = re.sub(r'http\S+', '', tweet)
        # Remove mentions
        tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)
        # Remove hashtags
        tweet = re.sub(r'#', '', tweet)
        # Remove non-alphabetic characters
        tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)
        # Convert to lowercase
        tweet = tweet.lower()
        # Remove stop words
        stop_words = set(stopwords.words('english'))
        words = tweet.split()
        filtered_words = [word for word in words if word not in stop_words]
        return ' '.join(filtered_words)

def splitScore(df):
    df['negative'] = df['Score'].apply(lambda x: x.get('neg'))
    df['neutral'] = df['Score'].apply(lambda x: x.get('neu'))
    df['positive'] = df['Score'].apply(lambda x: x.get('pos'))
    df['compound'] = df['Score'].apply(lambda x: x.get('compound'))
    return df

def weighted_avg(df):
    weights = df['Like_Count'] + 1
    return (df['compound'] * weights).sum() / weights.sum()

#fill missing dates with no Weighted_Compound_Score with 0
def fillMissing(aggregated_data):
    tickers = ['dbs',
                 'garena',
                 'shopee',
                 'ocbc',
                 'uob',
                 'singtel',
                 'wilmar',
                 'singapore airlines',
                 'grab',
                 'genting',
                 'capitaland',
                 'great eastern',
                 'jardine',
                 'ascendas',
                 'st engineering',
                 'keppel']

    tickerMapper = {'dbs': 'd05.SI',
                     'garena': 'SE',
                     'shopee': 'SE',
                     'ocbc': '039.SI',
                     'uob': 'U11.SI',
                     'singtel': 'Z74.SI',
                     'wilmar': 'F34.SI',
                     'singapore airlines': 'C6L.SI',
                     'grab': 'GRAB',
                     'genting': 'G13.SI',
                     'capitaland': 'C38U.SI',
                     'great eastern': 'G07.SI',
                     'jardine': 'C07.SI',
                     'ascendas': 'A17U.SI',
                     'st engineering': 'S63.SI',
                     'keppel': 'BN4.SI'}

    dates = pd.date_range(start='2023-02-18', end='2023-04-19', freq='D') # date range of concern
    df1 = pd.DataFrame(list(np.ndindex((len(tickers), len(dates)))), columns=['Ticker_idx', 'Date_idx'])
    df1['Ticker_Name'] = df1['Ticker_idx'].apply(lambda x: tickers[x])
    df1['Date'] = df1['Date_idx'].apply(lambda x: dates[x])
    df1 = df1.drop(['Ticker_idx', 'Date_idx'], axis=1)

    df1['Ticker'] = df1['Ticker_Name'].apply(lambda x: tickerMapper[x])
    merged_data = pd.merge(df1, aggregated_data, on = ['Ticker_Name', 'Ticker', 'Date'], how = 'left')
    merged_data['Weighted_Compound_Score'] = merged_data['Weighted_Compound_Score'].fillna(0)
    return merged_data

In [3]:
#data here are those that are successfully scraped previously by snscrape from 18/2/2023-19/4/2023
raw_data = pd.read_csv('raw_data.csv')
raw_data['Datetime'] = pd.to_datetime(raw_data['Datetime'])
raw_data

,Ticker_Name,Ticker,Texts,Like_Count,Datetime
0,dbs,d05.SI,@MrRich @saifyakhtar @Uber @GrabMY @pertamadig...,1,2023-02-18 05:43:20
1,dbs,d05.SI,"@idcrypto7 Hi, we are replying from DBS India....",0,2023-02-18 09:00:00
2,dbs,d05.SI,"@shimshiper Hi, we are replying from DBS India...",0,2023-02-19 04:20:01
3,dbs,d05.SI,I have not yet checked my bank on my other dev...,0,2023-02-20 10:46:10
4,dbs,d05.SI,"@JOngkowidjaja Hi, we are replying from DBS In...",0,2023-02-21 04:25:00
...,...,...,...,...,...
9326,keppel,BN4.SI,One of my friends from high school was murdere...,3,2023-04-19 12:35:33
9327,keppel,BN4.SI,He truly thinks his BS will be swallowed whole...,0,2023-04-19 14:53:05
9328,keppel,BN4.SI,He truly thinks his BS will be swallowed whole...,0,2023-04-19 14:53:05
9329,keppel,BN4.SI,Doing what they do the best……stirring the pot ...,0,2023-04-19 22:03:46


In [4]:
processed_data = raw_data.copy()
processed_data['Texts_Cleaned'] = processed_data['Texts'].apply(lambda x: clean_tweet(x))
processed_data

,Ticker_Name,Ticker,Texts,Like_Count,Datetime,Texts_Cleaned
0,dbs,d05.SI,@MrRich @saifyakhtar @Uber @GrabMY @pertamadig...,1,2023-02-18 05:43:20,managed catch last minutes interview agree tak...
1,dbs,d05.SI,"@idcrypto7 Hi, we are replying from DBS India....",0,2023-02-18 09:00:00,hi replying dbs india please share details us ...
2,dbs,d05.SI,"@shimshiper Hi, we are replying from DBS India...",0,2023-02-19 04:20:01,hi replying dbs india please share details us ...
3,dbs,d05.SI,I have not yet checked my bank on my other dev...,0,2023-02-20 10:46:10,yet checked bank device ive resting recent hea...
4,dbs,d05.SI,"@JOngkowidjaja Hi, we are replying from DBS In...",0,2023-02-21 04:25:00,hi replying dbs india query pertaining dbs ind...
...,...,...,...,...,...,...
9326,keppel,BN4.SI,One of my friends from high school was murdere...,3,2023-04-19 12:35:33,one friends high school murdered going wrong h...
9327,keppel,BN4.SI,He truly thinks his BS will be swallowed whole...,0,2023-04-19 14:53:05,truly thinks bs swallowed whole hes stupid can...
9328,keppel,BN4.SI,He truly thinks his BS will be swallowed whole...,0,2023-04-19 14:53:05,truly thinks bs swallowed whole hes stupid can...
9329,keppel,BN4.SI,Doing what they do the best……stirring the pot ...,0,2023-04-19 22:03:46,beststirring pot chaos amp confusion humans sh...


In [5]:
sia = SentimentIntensityAnalyzer()
processed_data['Score'] = processed_data['Texts_Cleaned'].apply(lambda x: sia.polarity_scores(x))
processed_data = splitScore(processed_data)
processed_data.drop(columns = {'Score'}, inplace = True)
processed_data['Date'] = processed_data['Datetime'].apply(lambda x: x.date())
processed_data

,Ticker_Name,Ticker,Texts,Like_Count,Datetime,Texts_Cleaned,negative,neutral,positive,compound,Date
0,dbs,d05.SI,@MrRich @saifyakhtar @Uber @GrabMY @pertamadig...,1,2023-02-18 05:43:20,managed catch last minutes interview agree tak...,0.064,0.675,0.261,0.7506,2023-02-18
1,dbs,d05.SI,"@idcrypto7 Hi, we are replying from DBS India....",0,2023-02-18 09:00:00,hi replying dbs india please share details us ...,0.000,0.781,0.219,0.5574,2023-02-18
2,dbs,d05.SI,"@shimshiper Hi, we are replying from DBS India...",0,2023-02-19 04:20:01,hi replying dbs india please share details us ...,0.000,0.712,0.288,0.7269,2023-02-19
3,dbs,d05.SI,I have not yet checked my bank on my other dev...,0,2023-02-20 10:46:10,yet checked bank device ive resting recent hea...,0.172,0.662,0.166,-0.0531,2023-02-20
4,dbs,d05.SI,"@JOngkowidjaja Hi, we are replying from DBS In...",0,2023-02-21 04:25:00,hi replying dbs india query pertaining dbs ind...,0.000,0.797,0.203,0.5574,2023-02-21
...,...,...,...,...,...,...,...,...,...,...,...
9326,keppel,BN4.SI,One of my friends from high school was murdere...,3,2023-04-19 12:35:33,one friends high school murdered going wrong h...,0.403,0.430,0.167,-0.6597,2023-04-19
9327,keppel,BN4.SI,He truly thinks his BS will be swallowed whole...,0,2023-04-19 14:53:05,truly thinks bs swallowed whole hes stupid can...,0.284,0.532,0.184,-0.5859,2023-04-19
9328,keppel,BN4.SI,He truly thinks his BS will be swallowed whole...,0,2023-04-19 14:53:05,truly thinks bs swallowed whole hes stupid can...,0.284,0.532,0.184,-0.5859,2023-04-19
9329,keppel,BN4.SI,Doing what they do the best……stirring the pot ...,0,2023-04-19 22:03:46,beststirring pot chaos amp confusion humans sh...,0.246,0.478,0.276,0.2263,2023-04-19


In [6]:
aggregated_data = processed_data.groupby(['Ticker_Name', 'Ticker', 'Date'], as_index = False).apply(weighted_avg)
aggregated_data.rename(columns = {None: 'Weighted_Compound_Score'}, inplace = True)
aggregated_data['Date'] = pd.to_datetime(aggregated_data['Date'] )
aggregated_data = fillMissing(aggregated_data)
aggregated_data

,Ticker_Name,Date,Ticker,Weighted_Compound_Score
0,dbs,2023-02-18,d05.SI,0.686200
1,dbs,2023-02-19,d05.SI,0.726900
2,dbs,2023-02-20,d05.SI,-0.053100
3,dbs,2023-02-21,d05.SI,0.110959
4,dbs,2023-02-22,d05.SI,0.120400
...,...,...,...,...
971,keppel,2023-04-15,BN4.SI,-0.034180
972,keppel,2023-04-16,BN4.SI,0.017633
973,keppel,2023-04-17,BN4.SI,0.063433
974,keppel,2023-04-18,BN4.SI,-0.119913


In [7]:
key = r'C:\Users\hsinz\Desktop\nus\Y3S2\IS3107\Proj\sharedkey.json'

In [8]:
openfile=open(key)
jsondata=json.load(openfile)
openfile.close()

In [9]:
credentials_path = key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= credentials_path
client = bigquery.Client()
project_id = jsondata['project_id']
staging_table_id = project_id + ".twitter_data.stock_info"
process_table_id = project_id + ".twitter_data.stock_processed"
aggregated_table_id = project_id + ".twitter_data.stock_aggregated"

In [10]:
def pushData():  
    job_config = bigquery.LoadJobConfig(
        encoding='UTF-8',
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
    )
    job = client.load_table_from_dataframe(raw_data, staging_table_id, job_config=job_config)
    job = client.load_table_from_dataframe(processed_data, process_table_id, job_config=job_config)
    job = client.load_table_from_dataframe(aggregated_data, aggregated_table_id, job_config=job_config)

    job.result()

In [11]:
pushData()